In [10]:
# đọc file
filename='train.txt'
lines=[]
count=0
#Max=-1
Max=10000
with open(filename,'r') as f:
    for s in f:
        count+=1
        if count>Max and Max!=-1:
            break
        lines.append(s.strip())
print(len(lines))
print(lines[:5])          

9797
["Here are two reasons companies fail: they only do more of the same, or they only do what's new", 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation', 'Both are necessary, but it can be too much of a good thing', 'Consider Facit', "I'm actually old enough to remember them"]


In [11]:
# tokenize sentences 
import nltk
sentences=[]
all_tokens_count=0
for line in lines:
    tokens = nltk.word_tokenize(line.lower())
    all_tokens_count+=len(tokens)
    #sentences.append(tokens)
    sentences.append(['<s>']+tokens+['</s>'])
print('all_tokens_count=',all_tokens_count)
print(len(sentences))
print(sentences[:5])

all_tokens_count= 182091
9797
[['<s>', 'here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they', 'only', 'do', 'more', 'of', 'the', 'same', ',', 'or', 'they', 'only', 'do', 'what', "'s", 'new', '</s>'], ['<s>', 'to', 'me', 'the', 'real', ',', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', ':', 'exploration', 'and', 'exploitation', '</s>'], ['<s>', 'both', 'are', 'necessary', ',', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing', '</s>'], ['<s>', 'consider', 'facit', '</s>'], ['<s>', 'i', "'m", 'actually', 'old', 'enough', 'to', 'remember', 'them', '</s>']]


In [12]:
# counting 1-gram 
from collections import Counter
counter_unigram=Counter()
for sent in sentences:
    counter_unigram.update(sent)
V=len(counter_unigram)
print('V=',V)
n=0
for gram in counter_unigram:
    n+=counter_unigram[gram]
n=n-counter_unigram['<s>']-counter_unigram['</s>']
print('n=',n)
print(counter_unigram['the'])
print(counter_unigram['he'])

V= 12645
n= 182091
7960
389


In [13]:
from nltk import ngrams
bi_grams=[]
for sent in sentences:
    gram2=ngrams(sent,2)
    bi_grams.extend(gram2)
print(len(bi_grams))

for i in range(3):
    print(bi_grams[i])

counter_bigram = Counter(bi_grams)
print('V=',len(counter_bigram))
print(counter_bigram[('here','are')])

191888
('<s>', 'here')
('here', 'are')
('are', 'two')
V= 78653
10


In [14]:
tri_grams=[]
for sent in sentences:
    gram3=ngrams(sent,3)
    tri_grams.extend(gram3)
print(len(tri_grams))

for i in range(3):
    print(tri_grams[i])

counter_trigram = Counter(tri_grams)
print('V=',len(counter_trigram))
print(counter_trigram[('here','are','two')])

182091
('<s>', 'here', 'are')
('here', 'are', 'two')
('are', 'two', 'reasons')
V= 139716
2


In [15]:
# Backoff
# tính prob theo từng loại: 1-gram, 2-gram, 3-gram
def uni_prob(word):
    return max(1,counter_unigram[word])/all_tokens_count

def bi_prob(word1,word2):
    if counter_bigram[(word1,word2)]>0:
        return counter_bigram[(word1,word2)]/counter_unigram[word1]
    else:
        return 0.4*uni_prob(word2)
    
def tri_prob(word1,word2,word3):
    if counter_trigram[(word1,word2,word3)]>0:
        return counter_trigram[(word1,word2,word3)]/counter_bigram[(word1,word2)]
    else:
        return 0.4*bi_prob(word1,word2)


In [16]:
# tính xác suất của một câu, normalize theo 1 từ 
def probLM(sent,n):
    if n>3 or n<1: # không xét trường hợp này 
        return 0
    tokens=nltk.word_tokenize(sent.lower())
    tokens += ['<s>']+tokens
    prob=1
    for i in range(1,len(tokens)):
        if n==1:
            prob*=uni_prob(tokens[i])
        elif n==2:
            prob*=bi_prob(tokens[i-1],tokens[i])
        elif n==3:
            if i>=2:
                prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
            else:
                prob*=bi_prob(tokens[i-1],tokens[i])
    l=len(tokens)-1
    return prob**(1/l)

In [17]:
sent='the human body with new abilities is no longer a question'
#sent='the human body with new from abilities is no longer a question'
#sent='A few years back from'
print('n=1')
pr=probLM(sent,1)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=2')
pr=probLM(sent,2)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=3')
pr=probLM(sent,3)
print('prob=',pr)
print('perplexity=',1/pr)

n=1
prob= 0.001416170749153918
perplexity= 706.1295402390167
n=2
prob= 0.01841810474958468
perplexity= 54.29440290388997
n=3
prob= 0.15967529022070817
perplexity= 6.26270976941873


In [18]:
# kiểm tra xem 2 xâu có xác suất hơn nhau thế nào, ví dụ cho bài toán speech to text
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'
pr=probLM(sent1,3)
print('prob=',pr)
print('perplexity=',1/pr)

pr=probLM(sent2,3)
print('prob=',pr)
print('perplexity=',1/pr)

prob= 0.15967529022070817
perplexity= 6.26270976941873
prob= 0.0039286601661115295
perplexity= 254.53970506941815
